# Bioinformatics Assignment 1

### Phylogenetic Tree Construction by Sequence-Based Distance

In this Jupyter NoteBook, required function definitions are provided to you. You are free to add different functions but I will be using these methods to test your code. 

#### Reading file

In [78]:
import json
from pprint import pprint

1. Read "organisms.txt" file into "organisms" dictionary whose keys are the GenBank IDs and the values are the list of names and the protein sequences of *COX3* gene of the corresponding organism.

> 'NC_012920.1': ['Human', 'MTHQSHAYHMVKPSPWPLTGALSALLMTSGLAMWFHFHSMTLLMLGLLTNTLTMYQWWRD
VTRESTYQGHHTPPVQKGLRYGMILFITSEVFFFAGFFWAFYHSSLAPTPQLGGHWPPTGITPLNPLEVPLLNTSVLLASGVSITW
AHHSLMENNRNQMIQALLITILLGLYFTLLQASEYFESPFTISDGIYGSTFFVATGFHGLHVIIGSTFLTICFIRQLMFHFTSKHH
FGFEAAAWYWHFVDVVWLFLYVSIYWWGS']

In [53]:
# Read organisms.txt to dictionary.
with open('organisms.txt') as f:
    organisms = json.loads(f.read().replace("'","\""))

organisms

{'NC_012920.1': ['Human',
  'MTHQSHAYHMVKPSPWPLTGALSALLMTSGLAMWFHFHSMTLLMLGLLTNTLTMYQWWRDVTRESTYQGHHTPPVQKGLRYGMILFITSEVFFFAGFFWAFYHSSLAPTPQLGGHWPPTGITPLNPLEVPLLNTSVLLASGVSITWAHHSLMENNRNQMIQALLITILLGLYFTLLQASEYFESPFTISDGIYGSTFFVATGFHGLHVIIGSTFLTICFIRQLMFHFTSKHHFGFEAAAWYWHFVDVVWLFLYVSIYWWGS'],
 'NC_001700.1': ['Cat',
  'MTHQTHAYHMVNPSPWPLTGALSALLMTSGLAMWFHYNLTLLLTLGMTTNLLTMYQWWRDIIRESTFQGHHTPIVQKGLRYGMILFIISEVFFFAGFFWAFYHSSLAPTPELGGCWPPTGIIPLNPLEVPLLNTSVLLASGVSITWAHHSLMEGNRKHMLQALFITISLGVYFTLLQASEYYETSFTISDGVYGSTFFMATGFHGLHVIIGSTFLIVCFLRQLKYHFTSNHHFGFEAAAWYWHFVDVVWLFLYVSIYWWGS'],
 'KT901460.1': ['Dog',
  'MTHQTHAYHMVNPSPWPLTGALSALLMTSGLIMWFHYNSMSLLTLGLTTNLLTMYQWWRDVIREGTFQGHHTPIVQKGLRYGMVLFIVSEVFFFAGFFWAFYHSSLAPTPELGGCWPPTGIIPLNPLEVPLLNTSVLLASGVSITWAHHSLMEGNRKHMLQALFITISLGVYFTLLQASEYYETSFTISDGVYGSTFFMATGFHGLHVIIGSTFLIVCFLRQLHYHFTSNHHFGFEAAAWYWHFVDVVWLFLYVSIYWWGS'],
 'DQ874614.2': ['House mouse',
  'MTHQTHAYHMVNPSPWPLTGAFSALLLTSGLVMWFHYNSITLLTLGLLTNILTMYQWWRDVIREGTYQGHHTPIVQKGLRYGMILFIVS

In [80]:
with open('blosum62_ncbi.txt') as f:
    data = f.read().splitlines()

BLOSUM = {}
all_chars = data[0].split()

for char in all_chars:
    BLOSUM[char] = {}

for line in data[1:]:
    nums = line.split()[1:]
    temp_dict = {}
    
    for num, char in zip(nums, all_chars):
        temp_dict[char] = int(num)
        
    BLOSUM[line.split()[0]] = temp_dict
    
BLOSUM

{'A': {'A': 4,
  'R': -1,
  'N': -2,
  'D': -2,
  'C': 0,
  'Q': -1,
  'E': -1,
  'G': 0,
  'H': -2,
  'I': -1,
  'L': -1,
  'K': -1,
  'M': -1,
  'F': -2,
  'P': -1,
  'S': 1,
  'T': 0,
  'W': -3,
  'Y': -2,
  'V': 0,
  'B': -2,
  'Z': -1,
  'X': 0,
  '*': -4},
 'R': {'A': -1,
  'R': 5,
  'N': 0,
  'D': -2,
  'C': -3,
  'Q': 1,
  'E': 0,
  'G': -2,
  'H': 0,
  'I': -3,
  'L': -2,
  'K': 2,
  'M': -1,
  'F': -3,
  'P': -2,
  'S': -1,
  'T': -1,
  'W': -3,
  'Y': -2,
  'V': -3,
  'B': -1,
  'Z': 0,
  'X': -1,
  '*': -4},
 'N': {'A': -2,
  'R': 0,
  'N': 6,
  'D': 1,
  'C': -3,
  'Q': 0,
  'E': 0,
  'G': 0,
  'H': 1,
  'I': -3,
  'L': -3,
  'K': 0,
  'M': -2,
  'F': -3,
  'P': -2,
  'S': 1,
  'T': 0,
  'W': -4,
  'Y': -2,
  'V': -3,
  'B': 3,
  'Z': 0,
  'X': -1,
  '*': -4},
 'D': {'A': -2,
  'R': -2,
  'N': 1,
  'D': 6,
  'C': -3,
  'Q': 0,
  'E': 2,
  'G': -1,
  'H': -1,
  'I': -3,
  'L': -4,
  'K': -1,
  'M': -3,
  'F': -3,
  'P': -1,
  'S': 0,
  'T': -1,
  'W': -4,
  'Y': -3,
  'V': 

#### Algorithm implementation

2. Implement Needleman Wunsch algorithm using the [BLOSUM62](!https://www.ncbi.nlm.nih.gov/Class/FieldGuide/BLOSUM62.txt) substitution matrix.

In [104]:
import numpy as np

def needleman_wunsch(seq1, seq2):
    # 'U' is -1, 'L' is 1, 'D' is 0
    seq1, seq2 = '*'+seq1, '*'+seq2

    allignment_matrix = []
    way_matrix = []
    for i in range(len(seq1)):
        temp = []
        for j in range(len(seq2)):
            temp.append(None)
        allignment_matrix.append(temp)
        way_matrix.append(temp)
    
    allignment_matrix[0][0] = BLOSUM['*']['*']
    
    # fill the first column
    for i in range(1, len(allignment_matrix)):
        allignment_matrix[i][0] = allignment_matrix[i-1][0] + BLOSUM[seq1[i]]['*']
        way_matrix[i][0] = -1
    
    # fill the first row
    for j in range(1, len(allignment_matrix[0])):
        allignment_matrix[0][j] = allignment_matrix[0][j-1] + BLOSUM[seq2[j]]['*']
        way_matrix[0][j] = 1
    
    for i in range(1, len(allignment_matrix)):
        for j in range(1, len(allignment_matrix[0])):
            upper_score = allignment_matrix[i-1][j] + BLOSUM[seq1[i]]['*']
            left_score = allignment_matrix[i][j-1] + BLOSUM[seq2[j]]['*']
            diag_score = allignment_matrix[i-1][j-1] + BLOSUM[seq1[i]][seq2[j]]
            cell_score = max(upper_score, left_score, diag_score)
            if upper_score == cell_score:
                way_matrix[i][j] = -1
            elif left_score == cell_score:
                way_matrix[i][j] = 1
            else:
                way_matrix[i][j] = 0
            allignment_matrix[i][j] = cell_score
    
    best_alignment_score_needleman_wunsch = allignment_matrix[len(allignment_matrix)-1][len(allignment_matrix[0])-1]
    best_alignment_needleman_wunsch = way_matrix
    # Find the alignment with the best score.
    return best_alignment_score_needleman_wunsch, best_alignment_needleman_wunsch



seq1 = organisms['EF153719.1'][1] # Turkey organism
seq2 = organisms['KM096864.1'][1] # Chicken organism

#seq1 = seq1[:10]
#print(seq1)
#seq2 = seq2[:10]
#print(seq2)

# Functions tests
best_alignment_score_needleman_wunsch, best_alignment_needleman_wunsch = needleman_wunsch(seq1, seq2)
print(best_alignment_needleman_wunsch)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [-1, 6, 2, -1, 1, 0, -1, 0, 0, -1, 6, 2, -2, -1, 0, -1, 0, -1, 2, 1, -2, 0, 0, 0, 0, 3, 3, 0, 0, 0, -2, 3, 2, 6, 2, 1, -1, 0, 0, 0, 0, 0, 3, 3, 0, 6, 2, 3, 3, 0, 6, 3, 3, 2, 6, 3, 1, 0, 0, 0, -2, 2, 2, 0, -1, 0,

3. Implement Smith Waterman algorithm using the [BLOSUM62](!https://www.ncbi.nlm.nih.gov/Class/FieldGuide/BLOSUM62.txt) substitution matrix.

In [ ]:
def smith_waterman(seq1, seq2):
    # Find the alignment with the best score.
    return best_alignment_score_smith_waterman, best_alignment_smith_waterman

#### Calculate Score Matrix

4. Using Needleman Wunsch and Smith Waterman algorithms you will find the **scores**, i.e. similarities between any two sequences. Now, crete an NxN score matrix where N is the length of organisms dictionary and calculate the scores of each organism pairs.

In [ ]:
# Calculate the score matrices for organism pairs using needleman_wunsch and smith_waterman algorithms.
def calculate_score_matrix(organisms, algorithm):
    # Calculate score matrix using smith_waterman() or needleman_wunsch() 
    return score_matrix

#### Calculate Distance Matrix

5. Now, you will use the score matrices to calculate **distances** between organism pairs. To do that;
    1. Find the maximum values of the score matrices.
    2. Subtract the each element of these matrices from the corresponding maximum values.
    3. Store resulting NxN matrices.

In [ ]:
# Calculate the distances between organism pairs.
def calculate_distance_matrix(score_matrix, organisms, algorithm):
    # Calculate distance matrix of smith waterman score matrix or needleman wunsch score matrix
    return distance_matrix

#### Generate Phylogenetic Tree

6. Finally, use below code to generate phylogenetic trees according to both Needleman Wunsch and Smith Waterman distance matrices.

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram

def generate_phylogenetic_tree(organisms, distance_matrix, algorithm):    
    average = linkage(distance_matrix, "average")
    dendrogram(average, labels=list(organisms.keys()), orientation="left", leaf_font_size=10)
    pylab.subplots_adjust(bottom=0.1, left=0.2, right=1.0, top=1.0)
    # Save figure as pylab.savefig("YourNameSurname" + algorithm +".jpg")
    # Show figure

#### Function Calls

In [ ]:
algorithm = 'NW'
seq1 = organisms['EF153719.1'][1] # Turkey organism
seq2 = organisms['KM096864.1'][1] # Chicken organism

# Functions tests
best_alignment_score_needleman_wunsch, best_alignment_needleman_wunsch = needleman_wunsch(seq1, seq2)
best_alignment_score_smith_waterman, best_alignment_smith_waterman = smith_waterman(seq1, seq2)

# Phylogenetic tree test
score_matrix = calculate_score_matrix(organisms, algorithm)
distance_matrix = calculate_distance_matrix(score_matrix, organisms, algorithm)
generate_phylogenetic_tree(organisms, distance_matrix)

#### Comments

7. Comment on the results of two output images.

Notes: 
1. Your are not allowed to use Numpy.
2. You are not allowed to use any libraries to find the Needleman Wunsch and Smith Waterman scores.
3. You can only use standard libraries apart from the given codes.
4. Please submit your assignment using Moodle. Upload a single zip file named as YourNameSurname.zip. Your zip file should include your report, your source code, and the corresponding read.me file. You can use any programming language of your choice. But, your read.me file should clearly explain how to run your program.
5. For any question e-mail me from selen.parlar@boun.edu.tr